In [60]:
from pdf2image import convert_from_path
import pytesseract

# Path to the PDF
pdf_path = "Samples/Contract 13.pdf"


def extract_text_from_pdf(pdf_path):
    images = convert_from_path(pdf_path)
    text = ""
    for image in images:
        text += pytesseract.image_to_string(image)
    return text


# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text)  # Print first 500 characters for verification

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

EXHIBIT A
HOSPITAL SERVICES —- GROUP COMMERCIAL CONTRACT and MEDICARE

In Accordance with Section 2.1, Hospital Services include all hospital services provided
by HOSPITAL and include, but are not limited to, those specified below: CODES: 1 — Provided
by HOSPITAL; 2-— Arranged by HOSPITAL; N/A — Not Available.

HOSPITAL SERVICES CODE HOSPITAL SERVICES CODE HOSPITAL SERVICES | CODE
MEDICAL SERVICES SPECIAL SERVICES PEDIATRIC SERVICES
Medical/Surgical Acute Care Services Burn Care Unit Pediatric Acute Care
Medical/Surgical Intensive Care Unit Chemotherapy Pediatric Intensive Care
Step-down/Transitional Care Unite { Gastrointestinal Laboratory Pediatric Oncology
Other: Hemodialysis - Inpatient Pediatric Surgery
ANCILLARY SERVICES Hemodialysis -Outpatient Other:
Electrocardiography Hyperbaric Chamber Services PERINATAL/DELIVERY SVCS
Electroencephalography | Immunosupressed (AIDS) Labor/Delivery Room Services
Unit
Electromyography Infusion Therapy Alternative Birthing Center
Pharmacy Servic

In [61]:
print(pdf_text)

EXHIBIT A
HOSPITAL SERVICES —- GROUP COMMERCIAL CONTRACT and MEDICARE

In Accordance with Section 2.1, Hospital Services include all hospital services provided
by HOSPITAL and include, but are not limited to, those specified below: CODES: 1 — Provided
by HOSPITAL; 2-— Arranged by HOSPITAL; N/A — Not Available.

HOSPITAL SERVICES CODE HOSPITAL SERVICES CODE HOSPITAL SERVICES | CODE
MEDICAL SERVICES SPECIAL SERVICES PEDIATRIC SERVICES
Medical/Surgical Acute Care Services Burn Care Unit Pediatric Acute Care
Medical/Surgical Intensive Care Unit Chemotherapy Pediatric Intensive Care
Step-down/Transitional Care Unite { Gastrointestinal Laboratory Pediatric Oncology
Other: Hemodialysis - Inpatient Pediatric Surgery
ANCILLARY SERVICES Hemodialysis -Outpatient Other:
Electrocardiography Hyperbaric Chamber Services PERINATAL/DELIVERY SVCS
Electroencephalography | Immunosupressed (AIDS) Labor/Delivery Room Services
Unit
Electromyography Infusion Therapy Alternative Birthing Center
Pharmacy Servic

In [46]:
def chunk_text(text, chunk_size=500):
    return [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]


# Chunk the extracted PDF text
chunks = chunk_text(pdf_text)
print(f"Number of chunks: {len(chunks)}")

Number of chunks: 61


In [47]:
from sentence_transformers import SentenceTransformer


# Initialize the MiniLM model

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


def get_embeddings(model, texts):
    return model.encode(texts)



# Generate embeddings for the chunks
chunk_embeddings = get_embeddings(model, chunks)

print(f"Number of embeddings: {len(chunk_embeddings)}")

/Users/krishkhadria/anaconda3/envs/hilabs/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Number of embeddings: 61


In [55]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Initialize the HuggingFace embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# Assuming 'chunks' is a list of text chunks
documents = [Document(page_content=chunk) for chunk in chunks]

# Create the Chroma collection
chroma_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="data",  # Specify where to save the collection
    collection_name="lc_chroma_demo",  # Choose a name for your collection
)

# Now let's search for relevant chunks based on a user query
user_query = "what is PHCMC rate for intermediate care? Provide answer only in not more than 20 words"
relevant_results = chroma_db.similarity_search(query=user_query, k=5)
print(relevant_results)

/Users/krishkhadria/anaconda3/envs/hilabs/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[Document(page_content='ll for Covered Services\npursuant to this Agreement.\n\nVought Health shall pay Hospital the rates below less any Copayments and deductibles.\n\n1\n\nHospital Services:\nInpatient 100% of the Medicare DRG rates or 100% of the\ncurrent Medicare Fee Schedule\n\nOutpatient/Emergency Room 100% of the Medicare Allowable rates (APC rates)\n\nOutpatient Surgery or 100% of the Current Medicare Fee Schedule\nObservation whichever is applicable.\nOther Outpatient\n\nAny Medically Necessary Covered Services not otherwise p'), Document(page_content='ll for Covered Services\npursuant to this Agreement.\n\nVought Health shall pay Hospital the rates below less any Copayments and deductibles.\n\n1\n\nHospital Services:\nInpatient 100% of the Medicare DRG rates or 100% of the\ncurrent Medicare Fee Schedule\n\nOutpatient/Emergency Room 100% of the Medicare Allowable rates (APC rates)\n\nOutpatient Surgery or 100% of the Current Medicare Fee Schedule\nObservation whichever is appl

In [56]:
# print(relevant_results[0])

In [57]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# # Load the FLAN-T5 model and tokenizer
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [58]:
# def generate_answer_flant5(model, tokenizer, query, relevant_chunks):
#     # Combine the query with the relevant chunks to provide context
#     context = "\n".join([chunk.page_content for chunk in relevant_chunks])
#     input_text = f"Question: {query}\nContext: {context}\nAnswer:"

#     # Tokenize the input text
#     inputs = tokenizer(input_text, return_tensors="pt")

#     # Generate the answer using FLAN-T5
#     outputs = model.generate(**inputs, max_length=1024)

#     # Decode the generated answer
#     answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
#     return answer


# # Example usage
# # query = "What is the rate of reimbursement for inpatient services in this contract?"
# answer = generate_answer_flant5(model, tokenizer, user_query, relevant_results)
# print(f"Answer: {answer}")

In [59]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.1")

chain = prompt | model
context = "\n".join([chunk.page_content for chunk in relevant_results])
input_text = f"Task: use Context to answer the question asked.\n\n Context: {context}\n\n Question: {user_query} "
print(chain.invoke({"question": input_text}))

There is no mention of PHCMC rate or Intermediate Care in the provided context, specifically under Hospital Services.
